In [1]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer

stopwords = nltk.corpus.stopwords.words('portuguese')

texto = ['Recomendo a todos os meus familiares e amigos porque a compra foi fácil, o aplicativo é de simples manuseio, a devolução é rápida e o estorno é garantido.',
'Uma empresa confiável, na qual possui diversas opções de produtos.',
'Levando em consideração o preço do produto e a facilidade da compra.',
'Sempre indico, é a melhor empresa.',
'Recomendaria para amigos e familiares comprarem do site, apenas se não tiverem pressa em receber o produto em casa.',
'Certamente recomendaria, gostei da experiência.',
'Gostei muito de todo o processo, não há do que reclamar.',
'Entrega rápida, frete grátis e o produto chegou bem embalado.',
'Como disse anteriormente, a parte da compra achei uma loja eficiente e confiável, entretanto, o processo de devolução e reembolso foi moroso, (ainda não finalizado).',
'Recomendaria, pois foi fácil de achar o produto, e o preço foi bom também.',
'Produto com boa qualidade, entrega antes do prazo, mas se quiser cancelar vai ter um pouquinho de dor de cabeça, pois não se consegue cancelar pelo aplicativo. É preciso telefonar e ficar esperando por um bom tempo, sem contar que como não enviaram a nota fiscal, tive que descobrir como tira-la e ainda por cima imprimir.'
        ]

contagem = CountVectorizer(stop_words=stopwords)
matrix = contagem.fit_transform(texto)
adjacency_matrix = matrix.T * matrix

words = contagem.get_feature_names()

G = nx.Graph()
for palavra in words:
    G.add_node(palavra)

for i in range(len(words)):
    for j in range(i + 1, len(words)):
        weight = adjacency_matrix[i, j]
        if weight > 0:
            G.add_edge(words[i], words[j], weight=weight)

# Criar DataFrame com as arestas
edges_list = []
for edge in G.edges(data=True):
    source, target, attr = edge
    weight = attr['weight']
    edges_list.append([source, target, weight])

df_edges = pd.DataFrame(edges_list, columns=['Origem', 'Destino', 'Peso'])

df_edges = df_edges.groupby(['Origem','Destino'], sort=False, as_index=False).sum()


# Create a graph from a pandas dataframe
G = nx.from_pandas_edgelist(df_edges, 
                            source = 'Origem', 
                            target = 'Destino', 
                            edge_attr = 'Peso', 
                            create_using = nx.Graph())

from pyvis.network import Network

node_degree = dict(G.degree)


C:\Users\marce\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\marce\anaconda3\lib\site-packages\numpy\.libs\libopenblas.4SP5SUA7CBGXUEOC35YP2ASOICYYEQZZ.gfortran-win_amd64.dll
C:\Users\marce\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
# Degree centrality
degree_dict = nx.degree_centrality(G)

degree_df = pd.DataFrame.from_dict(degree_dict, orient='index', columns=['centrality'])

betweenness_dict = nx.betweenness_centrality(G)
betweenness_df = pd.DataFrame.from_dict(betweenness_dict, orient='index', columns=['centrality'])

closeness_dict = nx.closeness_centrality(G)
closeness_df = pd.DataFrame.from_dict(closeness_dict, orient='index', columns=['centrality'])

# Save centrality measures
nx.set_node_attributes(G, degree_dict, 'degree_centrality')
nx.set_node_attributes(G, betweenness_dict, 'betweenness_centrality')
nx.set_node_attributes(G, closeness_dict, 'closeness_centrality')


import community as community_louvain
communities = community_louvain.best_partition(G)

nx.set_node_attributes(G, communities, 'group')

com_net = Network(notebook = True, width="1000px", height="700px", bgcolor='#222222', font_color='white')
com_net.from_nx(G)
com_net.show("witcher_communities.html")

witcher_communities.html
